In [12]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus, CSVClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence

import numpy as np

# This is a local path. The model is too large to add to
# the Git repository
path = '../flair-custom/resume2/'
model = SequenceTagger.load(path + 'best-model.pt')

from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler

text = 'George Washington and I went to Washington State'
s = Sentence(text)

def predict_one_sentence(text, word_index):
    """
    Inputs: text - string, containing the sentence to analyze.
            word_index - integer, the index of the word who's prediction
                         you want to explain.

    Return: list of probabilities for each possible tag.
    """
    sentence = Sentence(text)
    model.predict(sentence, return_probabilities_for_all_classes=True)

    w = sentence[word_index].text

    tag_probs = {'O': 0, 'geo': 0, 'tim': 0, 'org': 0, 'per': 0, 'gpe': 0, 'art': 0, 'eve': 0, 'nat': 0}

    # iterate over tags in sentences (sentences[i])
    # in tags, iterate over tags_proba_dist['ner'][j]
    for probs in sentence[word_idx].tags_proba_dist['ner']:
        tag = probs.data_point

        if tag in ['<START>', '<STOP>']:
            tag = 'O'
        if tag != 'O':
           tag = tag[2:]

        tag_probs[tag] += probs.value

    tag_prob_array = np.zeros((len(tag_probs)))

    prob_tot = 0
    for i,val in enumerate(tag_probs.values()):
        tag_prob_array[i] = val
        prob_tot += val

    # There is a small delta between 1.0 and
    # the sum of all probabilities. It seems to be
    # about 1e-8 or so, which could be because
    # the internal model is using floats, but
    # LIME is using doubles.
    tag_prob_array[0] += 1.0 - prob_tot

    return tag_prob_array



def get_predict_function(word_index):
    """
    Instantiates and returns the predict function needed by the TextExplainer fit function.

    We instantiate 'predict_fun' here so that it will get the value of 'word_idx'. 
    """
    def predict_func(texts):

        out = np.zeros((len(texts), 9))
        for i in range(len(texts)):
            out[i,:] = predict_one_sentence(texts[i], word_index)

        return out
    return predict_func

word_idx = 6
func = get_predict_function(word_idx)
sampler = MaskingTextSampler(
    replacement="UNK",
    max_replace=0.7,
    token_pattern=None,
    bow=False
)

samples, similarity = sampler.sample_near(text, n_samples=5)
print(samples)

te = TextExplainer(n_samples=5000, sampler=sampler, position_dependent=True, random_state=42)
te.fit(text, func)

tag_names = ['O', 'geo', 'tim', 'org', 'per', 'gpe', 'art', 'eve', 'nat']

#the explainer needs just the one instance text from texts list
explain = te.explain_prediction(target_names=tag_names,top_targets=3)
print("WORD TO EXPLAIN", s[word_idx])
explain

2022-08-19 05:35:18,111 loading file ../flair-custom/resume2/best-model.pt
2022-08-19 05:35:20,951 SequenceTagger predicts: Dictionary with 35 tags: O, S-geo, B-geo, E-geo, I-geo, S-tim, B-tim, E-tim, I-tim, S-org, B-org, E-org, I-org, S-per, B-per, E-per, I-per, S-gpe, B-gpe, E-gpe, I-gpe, S-art, B-art, E-art, I-art, S-eve, B-eve, E-eve, I-eve, S-nat, B-nat, E-nat, I-nat, <START>, <STOP>
('UNK Washington UNK UNK went UNK UNK State', 'UNK Washington and I went to Washington State', 'UNK UNK UNK I UNK to UNK UNK', 'UNK Washington UNK UNK UNK to UNK State', 'UNK Washington UNK I went to Washington State')
WORD TO EXPLAIN Token[6]: "Washington"


Explanation(estimator="SGDClassifier(alpha=0.001, loss='log', penalty='elasticnet',\n              random_state=RandomState(MT19937) at 0x7FA232200E40)", description=None, error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target='geo', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[6] Washington', weight=4.852944796322416, std=None, value=1.0), FeatureWeight(feature='[5] to', weight=3.2775661576802277, std=None, value=1.0), FeatureWeight(feature='[4] went', weight=0.8127981869412001, std=None, value=1.0), FeatureWeight(feature='[7] State', weight=0.5313848250374239, std=None, value=1.0), FeatureWeight(feature='[1] Washington', weight=0.24200455804738258, std=None, value=1.0), FeatureWeight(feature='[3] I', weight=0.09318696045549824, std=None, value=1.0)], neg=[FeatureWeight(feature='<BIAS>', weight=-6.7307856047615475, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.8526744422194474, score=3.079099879722601, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='George Washington and I went to Washington State', spans=[('Washington', [(7, 17)], 0.24200455804738258), ('I', [(22, 23)], 0.09318696045549824), ('went', [(24, 28)], 0.8127981869412001), ('to', [(29, 31)], 3.2775661576802277), ('Washington', [(32, 42)], 4.852944796322416), ('State', [(43, 48)], 0.5313848250374239)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=9.809885484484148, std=None, value=None)], neg=[FeatureWeight(feature='<BIAS>', weight=-6.7307856047615475, std=None, value=1.0)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='org', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[2] and', weight=0.4508173267622864, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=0.3978428575790612, std=None, value=1.0), FeatureWeight(feature='[4] went', weight=0.21613668765037555, std=None, value=1.0)], neg=[FeatureWeight(feature='[5] to', weight=-1.2272204902456068, std=None, value=1.0), FeatureWeight(feature='[6] Washington', weight=-1.0597309457455695, std=None, value=1.0), FeatureWeight(feature='[7] State', weight=-0.38431238759333547, std=None, value=1.0), FeatureWeight(feature='[3] I', weight=-0.36348361646468397, std=None, value=1.0)], pos_remaining=0, neg_remaining=0), proba=0.10916314091375318, score=-1.9699505680574727, weighted_spans=WeightedSpans(docs_weighted_spans=[DocWeightedSpans(document='George Washington and I went to Washington State', spans=[('and', [(18, 21)], 0.4508173267622864), ('I', [(22, 23)], -0.36348361646468397), ('went', [(24, 28)], 0.21613668765037555), ('to', [(29, 31)], -1.2272204902456068), ('Washington', [(32, 42)], -1.0597309457455695), ('State', [(43, 48)], -0.38431238759333547)], preserve_density=False, vec_name=None)], other=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=0.3978428575790612, std=None, value=1.0)], neg=[FeatureWeight(feature=<FormattedFeatureName 'Highlighted in text (sum)'>, weight=-2.367793425636534, std=None, value=None)], pos_remaining=0, neg_remaining=0)), heatmap=None), TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[FeatureWeight(feature='[7] State', weight=0.6296026389865822, std=None, value=1.0), FeatureWeight(feature='[1] Washington', weight=0.11681986352703565, std=None, value=1.0), FeatureWeight(feature='[3] I', weight=0.07705461984284297, std=None, value=1.0), FeatureWeight(feature='[5] to', weight=0.0706674815756288, std=None, value=1.0)], neg=[FeatureWeight(feature='[6] Washington', weight=-3.0961393438077716, std=None, value=1.0), FeatureWeight(feature='<BIAS>', weight=-0.8674517709815235, std=None, value=1.0), FeatureWeight(feature='[4] went', weight=-0.3522809369112303, std=None, value=1.0), FeatureWeight(feature='[0] George', weight=-0.24641459277385194, std=None, value=1.0), FeatureWeight(feature='[2] and', weight=-0.1355476600856564